In [45]:
# Developer Note: I tried to let it as simple as possible.
# Therefore there are no functions asking for the newest driver software or freeing memory beforehand, etc.
# The sole purpose of this program is to show one of the simplest ways to interact with an IDS camera via the uEye API.
# (XS cameras are not supported)
#---------------------------------------------------------------------------------------------------------------------------------------

#Libraries
from pyueye import ueye
import numpy as np
import cv2
import sys
import time
import os
import ctypes

In [46]:
#Variables
hCam = ueye.HIDS(0)             #0: first available camera;  1-254: The camera with the specified camera ID
sInfo = ueye.SENSORINFO()
cInfo = ueye.CAMINFO()
pcImageMemory = ueye.c_mem_p()
MemID = ueye.int()
rectAOI = ueye.IS_RECT()
pitch = ueye.INT()
nBitsPerPixel = ueye.INT(24)    #24: bits per pixel for color mode; take 8 bits per pixel for monochrome
channels = 3                    #3: channels for color mode(RGB); take 1 channel for monochrome
m_nColorMode = ueye.INT()		# Y8/RGB16/RGB24/REG32
bytes_per_pixel = int(nBitsPerPixel / 8)
#---------------------------------------------------------------------------------------------------------------------------------------
print("START")
print()

START



In [47]:
# Starts the driver and establishes the connection to the camera
nRet = ueye.is_InitCamera(hCam, None)
if nRet != ueye.IS_SUCCESS:
    print("is_InitCamera ERROR")

# Reads out the data hard-coded in the non-volatile camera memory and writes it to the data structure that cInfo points to
nRet = ueye.is_GetCameraInfo(hCam, cInfo)
if nRet != ueye.IS_SUCCESS:
    print("is_GetCameraInfo ERROR")

# You can query additional information about the sensor type used in the camera
nRet = ueye.is_GetSensorInfo(hCam, sInfo)
if nRet != ueye.IS_SUCCESS:
    print("is_GetSensorInfo ERROR")

nRet = ueye.is_ResetToDefault( hCam)
if nRet != ueye.IS_SUCCESS:
    print("is_ResetToDefault ERROR")

# Set display mode to DIB
nRet = ueye.is_SetDisplayMode(hCam, ueye.IS_SET_DM_DIB)

# Set the right color mode
if int.from_bytes(sInfo.nColorMode.value, byteorder='big') == ueye.IS_COLORMODE_BAYER:
    # setup the color depth to the current windows setting
    ueye.is_GetColorDepth(hCam, nBitsPerPixel, m_nColorMode)
    bytes_per_pixel = int(nBitsPerPixel / 8)
    print("IS_COLORMODE_BAYER: ", )
    print("\tm_nColorMode: \t\t", m_nColorMode)
    print("\tnBitsPerPixel: \t\t", nBitsPerPixel)
    print("\tbytes_per_pixel: \t\t", bytes_per_pixel)
    print()

elif int.from_bytes(sInfo.nColorMode.value, byteorder='big') == ueye.IS_COLORMODE_CBYCRY:
    # for color camera models use RGB32 mode
    m_nColorMode = ueye.IS_CM_BGRA8_PACKED
    nBitsPerPixel = ueye.INT(32)
    bytes_per_pixel = int(nBitsPerPixel / 8)
    print("IS_COLORMODE_CBYCRY: ", )
    print("\tm_nColorMode: \t\t", m_nColorMode)
    print("\tnBitsPerPixel: \t\t", nBitsPerPixel)
    print("\tbytes_per_pixel: \t\t", bytes_per_pixel)
    print()

elif int.from_bytes(sInfo.nColorMode.value, byteorder='big') == ueye.IS_COLORMODE_MONOCHROME:
    # for color camera models use RGB32 mode
    m_nColorMode = ueye.IS_CM_MONO8
    nBitsPerPixel = ueye.INT(8)
    bytes_per_pixel = int(nBitsPerPixel / 8)
    print("IS_COLORMODE_MONOCHROME: ", )
    print("\tm_nColorMode: \t\t", m_nColorMode)
    print("\tnBitsPerPixel: \t\t", nBitsPerPixel)
    print("\tbytes_per_pixel: \t\t", bytes_per_pixel)
    print()

else:
    # for monochrome camera models use Y8 mode
    m_nColorMode = ueye.IS_CM_MONO8
    nBitsPerPixel = ueye.INT(8)
    bytes_per_pixel = int(nBitsPerPixel / 8)
    print("else")

# Can be used to set the size and position of an "area of interest"(AOI) within an image
nRet = ueye.is_AOI(hCam, ueye.IS_AOI_IMAGE_GET_AOI, rectAOI, ueye.sizeof(rectAOI))
if nRet != ueye.IS_SUCCESS:
    print("is_AOI ERROR")

width = rectAOI.s32Width
#width  = 680
height = rectAOI.s32Height

# Prints out some information about the camera and the sensor
print("Camera model:\t\t", sInfo.strSensorName.decode('utf-8'))
print("Camera serial no.:\t", cInfo.SerNo.decode('utf-8'))
print("Maximum image width:\t", width)
print("Maximum image height:\t", height)
print()

#---------------------------------------------------------------------------------------------------------------------------------------

IS_COLORMODE_MONOCHROME: 
	m_nColorMode: 		 6
	nBitsPerPixel: 		 8
	bytes_per_pixel: 		 1

Camera model:		 UI122xLE-M
Camera serial no.:	 4103194750
Maximum image width:	 752
Maximum image height:	 480



In [48]:
# Allocates an image memory for an image having its dimensions defined by width and height and its color depth defined by nBitsPerPixel
nRet = ueye.is_AllocImageMem(hCam, width, height, nBitsPerPixel, pcImageMemory, MemID)
if nRet != ueye.IS_SUCCESS:
    print("is_AllocImageMem ERROR")
else:
    # Makes the specified image memory the active memory
    nRet = ueye.is_SetImageMem(hCam, pcImageMemory, MemID)
    if nRet != ueye.IS_SUCCESS:
        print("is_SetImageMem ERROR")
    else:
        # Set the desired color mode
        nRet = ueye.is_SetColorMode(hCam, m_nColorMode)



# Activates the camera's live video mode (free run mode)
nRet = ueye.is_CaptureVideo(hCam, ueye.IS_DONT_WAIT)
if nRet != ueye.IS_SUCCESS:
    print("is_CaptureVideo ERROR")
    


# Enables the queue mode for existing image memory sequences
nRet = ueye.is_InquireImageMem(hCam, pcImageMemory, MemID, width, height, nBitsPerPixel, pitch)
if nRet != ueye.IS_SUCCESS:
    print("is_InquireImageMem ERROR")
else:
    print("Press q to leave the programm")

counter=0
# directory for saving pictures
directory = r'C:\Users\...'
# Change the current directory  
# to specified/ Target - directory  
os.chdir(directory)

number = ueye.UINT()
# check current Frames per Second (Bildrate)
# eine Variable im erwarteten Format (???ctypes double???) deklarieren
dblFPS=ctypes.c_double(0)
ms = ueye.DOUBLE(20)

# PixelClock checken -> default 20
ret=ueye.is_PixelClock(hCam, ueye.IS_PIXELCLOCK_CMD_GET, number, ueye.sizeof(number))
print('PxCLK current:',ret, number)
# default Belichtungszeit - anbhängig von PixelClock
ret = ueye.is_Exposure(hCam, ueye.IS_EXPOSURE_CMD_GET_EXPOSURE, ms, ueye.sizeof(ms));
print('EXP:',ret,ms)
# bei Bedarf PixelClock ändern (FPS und Belichtungszeit automatisch angepasst)
ret = ueye.is_PixelClock(hCam, ueye.IS_PIXELCLOCK_CMD_SET, ueye.UINT(14), ueye.sizeof(number))
print('PxCLK set:',ret, number)
# PixelClock checken -> neue PixelClock
ret = ueye.is_PixelClock(hCam, ueye.IS_PIXELCLOCK_CMD_GET, number, ueye.sizeof(number))
print('PxCLK current:',ret, number)

    
# Continuous image display
while(nRet == ueye.IS_SUCCESS and counter<=999):

    # In order to display the image in an OpenCV window we need to...
    # ...extract the data of our image memory
    array = ueye.get_data(pcImageMemory, width, height, nBitsPerPixel, pitch, copy=False)

    # bytes_per_pixel = int(nBitsPerPixel / 8)

    # ...reshape it in an numpy array...
    frame = np.reshape(array,(height.value, width.value, bytes_per_pixel))

    # ...resize the image by a half
    #frame = cv2.resize(frame,(0,0),fx=0.5, fy=0.5)
    
    #---------------------------------------------------------------------------------------------------------------------------------------
    #Include image data processing here
    counter=counter+1 
    filename = str(counter)+'_TestImage.png'
    
    #...and finally display it
    cv2.imshow("SimpleLive_Python_uEye_OpenCV", frame)
    
    # Bild speichern 
    cv2.imwrite(filename, frame)
    
    print("##################################################################################")
    # check current PixelClock
    ret = ueye.is_PixelClock(hCam, ueye.IS_PIXELCLOCK_CMD_GET, number, ueye.sizeof(number))
    print('PxCLK current:',ret, number)
    
    # dblFPS der Funktion übergeben und in der Funktion wird der FPS-Wert der Variablen dblFPS zugewiesen 
    ueye.is_GetFramesPerSecond (hCam, dblFPS)
    # Variable auf Konsole printen
    print('FPS: ', dblFPS)
    
    # check current Exposure (Belichtungszeit)
    ret = ueye.is_Exposure(hCam, ueye.IS_EXPOSURE_CMD_GET_EXPOSURE, ms, ueye.sizeof(ms));
    print('EXP:',ret,ms)
    print("##################################################################################")
    
    time.sleep(0.1)

    # Press q if you want to end the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    #---------------------------------------------------------------------------------------------------------------------------------------

# Releases an image memory that was allocated using is_AllocImageMem() and removes it from the driver management
ueye.is_FreeImageMem(hCam, pcImageMemory, MemID)

# Disables the hCam camera handle and releases the data structures and memory areas taken up by the uEye camera
ueye.is_ExitCamera(hCam)

# Destroys the OpenCv windows
cv2.destroyAllWindows()

print()
print("END")

Press q to leave the programm
PxCLK current: 0 20
EXP: 0 33.24165
PxCLK set: 0 20
PxCLK current: 0 14
##################################################################################
PxCLK current: 0 14
FPS:  c_double(0.0)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.017720039765525)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.035360440901155)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(20.767914402963996)
EXP: 0 47.48807142857143
############

##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.047048572378696)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.023817883279968)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.039963306303992)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.03336944061754)
EXP: 0 47.48807142857143
##################################################################################
#################

##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.019089537117612)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.014010040493996)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.009197826808578)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.038104214353037)
EXP: 0 47.48807142857143
##################################################################################
################

##################################################################################
PxCLK current: 0 14
FPS:  c_double(20.9920419169093)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.057818452123996)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.05294183282701)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.029079010455657)
EXP: 0 47.48807142857143
##################################################################################
###################

##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.00354959988238)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.023596885143885)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(20.99649358557121)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.00606445080695)
EXP: 0 47.48807142857143
##################################################################################
###################

...

##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.01272951153809)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.032484672576796)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(20.996405415392886)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.027177627083006)
EXP: 0 47.48807142857143
##################################################################################
#################

##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.0024908954202)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.007167645600678)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.00606445080695)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.031246122363996)
EXP: 0 47.48807142857143
##################################################################################
###################

##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.037130535394972)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.01776421431394)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.038591087632042)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.018382677489733)
EXP: 0 47.48807142857143
##################################################################################
#################

##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.02735448545012)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.004696650170978)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.019884811031236)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.039033719259344)
EXP: 0 47.48807142857143
##################################################################################
#################

##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.03885666437343)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(20.99614090930087)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.045852599057568)
EXP: 0 47.48807142857143
##################################################################################
##################################################################################
PxCLK current: 0 14
FPS:  c_double(21.047004274646568)
EXP: 0 47.48807142857143
##################################################################################
##################

In [12]:
#Kamera wieder freigeben, falls noch nicht geschehen!!!!!

# Releases an image memory that was allocated using is_AllocImageMem() and removes it from the driver management
ueye.is_FreeImageMem(hCam, pcImageMemory, MemID)

# Disables the hCam camera handle and releases the data structures and memory areas taken up by the uEye camera
ueye.is_ExitCamera(hCam)

# Destroys the OpenCv windows
cv2.destroyAllWindows()

print()
print("END")


END
